### 목표
- 네이버 영화 리뷰 데이터셋을 이용해 긍정/부정 분류 모델을 만들어보자
- TF-IDF 방식을 적용해보자
- Konlpy 한국어 형태소 분석기를 설치하고 활용해보자

In [1]:
!pip install konlpy

#### 데이터 수집
- 네이버 댓글 데이터 로딩
- label(0:부정, 1:긍정)

In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv("data/ratings_train.txt", delimiter='\t') 
df_test = pd.read_csv("data/ratings_test.txt", delimiter='\t')
# delimiter : 파일에서 데이터 사이의 간격을 구분해주는 구분자
# (설정하지 않으면 ,가 디폴트)

In [4]:
pd.set_option('display.max_colwidth',None) # 컬럼의 너비를 무제한으로 출력

In [5]:
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


#### 데이터 전처리
- 결측치 처리

In [8]:
df_train[df_train['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [9]:
df_test[df_test['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


- 문제/정답, 학습/평가 데이터로 분리

In [12]:
# 
X_train = df_train['document']
y_train = df_train['label']
X_test = df_test['document']
y_test = df_test['label']

In [13]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(150000,) (150000,) (50000,) (50000,)


In [14]:
# 데이터중에서 중요한 단어를 선별하기 위해 Tfidf 벡터라이저를 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

# BOW 기법
# 1.CountVectorize(단순히 단어의 빈도수만 파악)
# 2.TF-IDF(단어의 빈도수 + 단어가 등장하는 문서의 개수 둘 다 파악해서 중요도를 판단)

In [15]:
X_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [16]:
# TfidfVectorizer에는 디폴트로 word(단어) 기준으로 토큰화를 시켜주는 기능이 탑재
tf_idf_vect = TfidfVectorizer()

In [17]:
# TF_IDF 방식으로 3개의 행의 문장들을 단어 모음(BOW)로 만들기(학습)
tf_idf_vect.fit(X_train[:3])

TfidfVectorizer()

In [18]:
# vocabulary_ : 단어 모음의 단어와 인덱스 라벨 확인(가나다순으로 0번 부터 시작)
tf_idf_vect.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [19]:
# 토큰화 된 단어만 보기
tf_idf_vect.get_feature_names()

['가볍지',
 '너무재밓었다그래서보는것을추천한다',
 '더빙',
 '목소리',
 '않구나',
 '오버연기조차',
 '진짜',
 '짜증나네요',
 '초딩영화줄',
 '포스터보고']

- TF-IDF 방식은 영어에 맞춰져 있어 한글 텍스트에는 토큰화가 제대로 이루어지지 않는다

#### 한글에 특화되어 있는 konlpy 형태소 분석기로 TF-IDF 방식을 적용해보자

In [20]:
# 한글을 형태소별로 잘 토큰화 할 수 있는 꼬꼬마(Kkma) 라이브러리 불러오기
from konlpy.tag import Kkma

In [21]:
kkma = Kkma()

In [22]:
X_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [23]:
# nouns : 명사만 토큰화를 시켜줌
kkma.nouns(X_train[0])

['더빙', '목소리']

In [24]:
# 입력문장이 들어왔을때 명사만 토큰화 시켜주는 함수를 선언
def myTokenizer(text) :
    return kkma.nouns(text)

In [25]:
# Kkma 라이브러리로 명사만 추출해서 TF-IDF 방식을 활용해보자
# tokenizer : 토큰의 단위(기준)을 지정, 디폴트값은 word
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer)

In [26]:
tf_idf_vect.fit(X_train[:3])
tf_idf_vect.vocabulary_

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

- Kkma로 토큰화를 시킨 후 TF-IDF 방식을 적용하니 그냥 TF-IDF만 적용했을 때보다 더 토큰화가 잘 이루어지는 것을 확인

### POS(Part Of Speech) Tagging 활용
- 문장을 토큰화 한 후 쪼개진 형태소에 품사를 부여하는 것
- 한글에서는 형용사와 동사에 감정이 많이 들어있어서 감성분석에 활용해보자

In [27]:
data= "먹는다 먹다 먹었다 이쁘다 아름답다 사진 모자"

kkma.morphs(data)

['먹', '는', '다', '먹', '다', '먹', '었', '다', '이쁘', '다', '아름답', '다', '사진', '모자']

In [28]:
kkma.pos(data)

[('먹', 'VV'),
 ('는', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('다', 'EFN'),
 ('이쁘', 'VA'),
 ('다', 'ECS'),
 ('아름답', 'VA'),
 ('다', 'ECS'),
 ('사진', 'NNG'),
 ('모자', 'NNG')]

In [29]:
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [30]:
# 학습을 위해 데이터프레임화
df = pd.DataFrame(kkma.pos(data), columns=['morph', 'tag'])
df

,morph,tag
0,먹,VV
1,는,EPT
2,다,ECS
3,먹,VV
4,다,ECS
5,먹,VV
6,었,EPT
7,다,EFN
8,이쁘,VA
9,다,ECS


In [31]:
# 데이터 프레임에서 tag 컬럼을 인덱스로 설정
df.set_index('tag', inplace=True)

In [32]:
df.loc[['VV', 'VA', 'NNG']]   # 동사(VV), 형용사(VA), 보통명사(NNG)

,morph
tag,
VV,먹
VV,먹
VV,먹
VA,이쁘
VA,아름답
NNG,사진
NNG,모자


In [33]:
labels = ['VV', 'VA', 'NNG']

# 데이터프레임의 인덱스에 있는 VV, VA, NNG값들을 중복포함하여 배열로 나타냄
df.loc[df.index.intersection(labels)]['morph'].values
# → TF-IDF 방식에서도 단어의 빈도수는 중요도에 영향을 미치기 때문에 같은
#    품사의 같은 단어라도 개수를 표시할 수 있게 출력

array(['먹', '먹', '먹', '이쁘', '아름답', '사진', '모자'], dtype=object)

In [34]:
# 입력 텍스트에 대해서 pos tagging을 진행하여 VV, VA, NNG값만 반환해주는 함수
def myTokenizer2(text) :
    df = pd.DataFrame(kkma.pos(text), columns=['morph', 'tag'])
    df.set_index('tag', inplace=True)
    # 품사가 VV(동사), VA(형용사), NNG(보통명사)일 경우
    if ('VV' in df.index) or ('VA' in df.index) or ('NNG' in df.index) : 
        labels = ['VV', 'VA', 'NNG']
        return df.loc[df.index.intersection(labels)]['morph'].values
    else :
        return[]

In [35]:
X_train[:2]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
Name: document, dtype: object

In [36]:
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)

In [37]:
tf_idf_vect.fit(X_train[:2])
tf_idf_vect.vocabulary_

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'아': 4,
 '짜증나': 9,
 '더빙': 1,
 '목소리': 2,
 '흠': 11,
 '포스터': 10,
 '보고': 3,
 '영화': 6,
 '줄': 8,
 '오버': 7,
 '연기': 5,
 '가볍': 0}

In [38]:
final_tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf_vect.fit(X_train[:5000])

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x000001E8555159D0>)

In [39]:
# 5000개의 리뷰 데이터의 동사, 형용사, 일반명만 추출하여 단어모음집을 만들어줌
final_tf_idf_vect.vocabulary_

{'아': 3305,
 '짜증나': 5038,
 '더빙': 1227,
 '목소리': 1860,
 '흠': 6112,
 '포스터': 5641,
 '보고': 2270,
 '영화': 3726,
 '줄': 4855,
 '오버': 3790,
 '연기': 3654,
 '가볍': 45,
 '재': 4484,
 '추천': 5229,
 '어': 3491,
 '보': 2268,
 '교도소': 486,
 '이야기': 4181,
 '재미': 4495,
 '점': 4617,
 '조정': 4748,
 '멀': 1765,
 '없': 3574,
 '익살': 4227,
 '스파이': 3081,
 '더': 1216,
 '맨': 1744,
 '커스틴': 5324,
 '돋보이': 1287,
 '늙': 1053,
 '보이': 2293,
 '하': 5738,
 '이쁘': 4162,
 '걸음마': 255,
 '초등학교': 5177,
 '학년': 5779,
 '생인': 2751,
 '용': 3912,
 '별': 2255,
 '반개': 2093,
 '떼': 1465,
 '아깝': 3309,
 '원작': 3999,
 '긴장감': 734,
 '살리': 2653,
 '욕': 3905,
 '이응': 4193,
 '경': 303,
 '길': 737,
 '우': 3926,
 '생활': 2755,
 '발': 2111,
 '납': 893,
 '감금': 99,
 '반복': 2100,
 '이': 4124,
 '드라마': 1374,
 '가족': 69,
 '사람': 2588,
 '모': 1813,
 '엿': 3698,
 '나오': 836,
 '낫': 896,
 '액션': 3454,
 '안': 3386,
 '있': 4349,
 '평점': 5627,
 '우드': 3929,
 '식': 3169,
 '화려': 6009,
 '낮': 901,
 '헐': 5907,
 '길들이': 738,
 '인피': 4272,
 '니트': 1067,
 '짱': 5047,
 '나': 812,
 '죽': 4842,
 '때': 1443,
 '눈물': 1029

In [40]:
len(final_tf_idf_vect.vocabulary_)

6145

In [41]:
# transform : 토큰화된 자연어들을 기계가 이해할 수 있게 수치화
X_train = final_tf_idf_vect.transform(X_train[:5000])
X_test = final_tf_idf_vect.transform(X_test[:5000])

#### 모델 적용(로지스틱 회귀)
- 1차식인 직선(y=wx+b)으로 빠른 속도로 예측을 진행할 수 있는 '분류' 모델
- 텍스트 마이닝 분야는 기본적으로 데이터인 단어나 형태소 등의 개수가 워낙 다양하고 많기 때문에 속도가 빠른 모델을 적용

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
logi = LogisticRegression()
logi.fit(X_train, y_train[:5000])
logi.score(X_test, y_test[:5000])

0.776